In [1]:
import os
os.chdir("../")

In [2]:
import numpy as np
import torch

In [3]:
from storage.har_datasets import STSDataset, StreamingTimeSeries, StreamingTimeSeriesCopy, WISDMDataset
from s3ts.dtw_layer.lightningstsds import LSTSDataset
from s3ts.dtw_layer.dtw_layer import DTWLayer
from s3ts.dtw_layer.wrapper import lmodel
from pytorch_lightning import Trainer
import numpy as np
import torch

In [4]:
# this dataset comes at 20hz
ds = WISDMDataset("./datasets/WISDM/", wsize=80, normalize=True, label_mapping=None)

In [5]:
data_split = {
    "train": lambda x: x<1000000,
    "val": lambda x: (x>=1000000) * (x<1050000),
    "test": lambda x: x>=1050000
}

dm = LSTSDataset(ds, data_split=data_split, batch_size=128, random_seed=42, num_workers=8)

In [6]:
len(dm.ds_train) + len(dm.ds_val) + len(dm.ds_test)

1096768

In [9]:
model = torch.nn.Sequential(
    DTWLayer(8, 3, 40, l_out=40, rho=0.01),
    torch.nn.Conv2d(in_channels=8, out_channels=32, kernel_size=5), # out 36x36
    torch.nn.ReLU(),
    torch.nn.MaxPool2d(kernel_size=2), # out 18x18
    torch.nn.Conv2d(in_channels=32, out_channels=64, kernel_size=5), # out 14x14
    torch.nn.ReLU(),
    torch.nn.MaxPool2d(kernel_size=2), # out 7x7
    torch.nn.Conv2d(in_channels=64, out_channels=128, kernel_size=5), # out 3x3 
    torch.nn.ReLU(),
    torch.nn.Flatten(),
    torch.nn.Linear(in_features=128*9, out_features=12)
)

In [10]:
trainer = Trainer(max_epochs=50, accelerator="auto")
model_lightning = lmodel(model, 12, lr=0.001)
trainer.fit(model=model_lightning, datamodule=dm)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name       | Type               | Params
--------------------------------------------------
0 | model      | Sequential         | 277 K 
1 | softmax    | Softmax            | 0     
2 | train_acc  | MulticlassAccuracy | 0     
3 | train_f1   | MulticlassF1Score  | 0     
4 | val_acc    | MulticlassAccuracy | 0     
5 | val_f1     | MulticlassF1Score  | 0     
6 | val_auroc  | MulticlassAUROC    | 0     
7 | test_acc   | MulticlassAccuracy | 0     
8 | test_f1    | MulticlassF1Score  | 0     
9 | test_auroc | MulticlassAUROC    | 0     
--------------------------------------------------
277 K     Trainable params
0         Non-trainable params
277 K     Total params
1.110     Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

Epoch 0:   2%|▏         | 182/7803 [00:31<21:58,  5.78it/s, v_num=53, train_loss_step=0.889]